# Further Feature Engineering

Continuing on from our previous session on EDA, this Notebook includes further code for performing feature engineering. Rather than following a specific process, instead we will include a few examples of different things you may want to do:

## Dealing with Categorical Data

Essentially all ML requires our data to be numerical (at the point of analysis) but quite often we get data which is text. While we could just ignore this, it is often desireable to apply some form of transformation to numerical values.

If our data is binary (one of two options) this is quite easy. E.g. let's consider a feature which is "hasPhD?" with a "yes" for people who have one and "no" for the rest. We can easily transform this with an if statement like below (using _np.where()_ for inline calculation of a whole DataFrame but it works the same as an if):

In [1]:
import pandas as pd
import numpy as np

# create example dataframe
df = pd.DataFrame({"a": [1,2,3,4,5], "hasPhD?": ["yes", "yes", "no", "no", "no"]})
print(df)

print("\n")

# change "hasPhD? to numerical values"
df["hasPhD?"] = np.where(df["hasPhD?"] == "yes", 1, 0)
print(df)

   a hasPhD?
0  1     yes
1  2     yes
2  3      no
3  4      no
4  5      no


   a  hasPhD?
0  1        1
1  2        1
2  3        0
3  4        0
4  5        0


This deals with the binary case - how about cases where there are multiple categories?

In [2]:
# create example dataframe
df = pd.DataFrame({"a": [1,2,3,4,5], "businessType": ["Hospitality", "IT", "Education", "IT", "Oil"]})
df

,a,businessType
0,1,Hospitality
1,2,IT
2,3,Education
3,4,IT
4,5,Oil


Clearly we can't apply the same trick. Of course, we _could_ assign 1 to "Hospitatlity", 2 to "IT", 3 to "Education" (etc.) ... but is it really meaningful? Could we say $ Hospitality + IT = Education $? Or $ 2 * Hospitality = IT $? These caclulations show such a scheme is not sensible. Instead we use dummy variables:

In [3]:
# generate binary values using get_dummies
df = pd.get_dummies(df, columns=["businessType"], prefix=["type_"])
df

,a,type__Education,type__Hospitality,type__IT,type__Oil
0,1,0,1,0,0
1,2,0,0,1,0
2,3,1,0,0,0
3,4,0,0,1,0
4,5,0,0,0,1


This then achieves our goal - we have each category represented and the data shown, but no issues with dodgy numerical representations 

## Splitting Features

Another common scenario is where we have a text field (typically) in one column and want to split it into two. Take this example:

In [4]:
# create example dataframe
df = pd.DataFrame({"a": [1,2,3,4,5], "degree": ["MSc eBM", "BA Art", "MSc CSM", "BSc CS", "MSc eBM"]})
df

,a,degree
0,1,MSc eBM
1,2,BA Art
2,3,MSc CSM
3,4,BSc CS
4,5,MSc eBM


We want to split this data into degree type and subject. There are lots of ways of doing this but we can do it using a deliminator:

In [5]:
df[['degree', 'subject']] = df['degree'].str.split(' ', 1, expand=True)
df

,a,degree,subject
0,1,MSc,eBM
1,2,BA,Art
2,3,MSc,CSM
3,4,BSc,CS
4,5,MSc,eBM


## Filling Missing Values

We have looked at one way of doing this in a previous Notebook - deleting the row - but in fact there are a few options in _pandas:_

In [6]:
# DataFrame with missing data
df = pd.DataFrame({"a": [1,2,3,4,None], "hasPhD?": [1, 0, None, 0, 0]})
print(df)
print("\n")

# drop rows with missing data
dfdrop = df.dropna() 
print("Drop rows")
print(dfdrop)
print("\n")

# replace NAs with column mean
dfcol = df.fillna(df.mean()) 
print("Column mean")
print(dfcol)
print("\n")

# row mean
dfrow = df.apply(lambda row: row.fillna(row.mean()), axis=1)
print("Row mean")
print(dfrow)

     a  hasPhD?
0  1.0      1.0
1  2.0      0.0
2  3.0      NaN
3  4.0      0.0
4  NaN      0.0


Drop rows
     a  hasPhD?
0  1.0      1.0
1  2.0      0.0
3  4.0      0.0


Column mean
     a  hasPhD?
0  1.0     1.00
1  2.0     0.00
2  3.0     0.25
3  4.0     0.00
4  2.5     0.00


Row mean
     a  hasPhD?
0  1.0      1.0
1  2.0      0.0
2  3.0      3.0
3  4.0      0.0
4  0.0      0.0


## Normalising Data

When using data as features in machine learning we often want to ensure they are on the same scale or otherwise the features that are on the largest scale  will have the most influence on the model (e.g. "a" is on a scale of 0 to 1; "b" is on a scale -100 to 10,000 ... "b" will be more influential). There are a few methods to do this:

In [7]:
df = pd.DataFrame({"a": [1,2,3,4,5], "salary": [100000, 100, 20000, 0, 123456]})
print(df)
print("\n")

# mean normalisation
df["salary"] = (df["salary"] - df["salary"].mean()) / df["salary"].std()
print("Mean normalised")
print(df)
print("\n")

# min-max normalisation (feature scaling)
df = pd.DataFrame({"a": [1,2,3,4,5], "salary": [100000, 100, 20000, 0, 123456]})
df["salary"] = (df["salary"] - df["salary"].min()) / (df["salary"].max() -df["salary"].min())
print("Min/max normalised")
print(df)
print("\n")

# robust scaling (via scikit-learn)
# Similar to min-max but uses IQR so is robust to outliers
from sklearn.preprocessing import RobustScaler

df = pd.DataFrame({"a": [1,2,3,4,5], "salary": [100000, 100, 20000, 0, 123456]})

scaler = RobustScaler()
df = scaler.fit_transform(df)
df = pd.DataFrame(df, columns=['a', 'salary'])

print(df)

   a  salary
0  1  100000
1  2     100
2  3   20000
3  4       0
4  5  123456


Mean normalised
   a    salary
0  1  0.873912
1  2 -0.828288
2  3 -0.489211
3  4 -0.829992
4  5  1.273579


Min/max normalised
   a    salary
0  1  0.810005
1  2  0.000810
2  3  0.162001
3  4  0.000000
4  5  1.000000


     a    salary
0 -1.0  0.800801
1 -0.5 -0.199199
2  0.0  0.000000
3  0.5 -0.200200
4  1.0  1.035596


## Aggregation and GroupBy

We use the groupby function when we want to change the unit of aggregation of a DataFrame. For instance, we may have a DataFrame that has 1x row per day and we want to aggregate up to 1x row per month:

In [8]:
df = pd.DataFrame({"Team": ["DS", "DT", "DT", "DS", "DS"], "salary": [100000, 100, 20000, 0, 100000]})
print(df)
print("\n")

# team based aggregation (average)
mean_df = df.groupby(['Team']).mean()
print("Aggregation by mean")
print(mean_df)
print("\n")

# team based aggregation (sum)
sum_df = df.groupby(['Team']).sum()
print("Aggregation by mean")
print(sum_df)
print("\n")

# team based aggregation (median)
median_df = df.groupby(['Team']).median()
print("Aggregation by Median")
print(median_df)
print("\n")

# team based aggregation (maximum)
max_df = df.groupby(['Team']).max()
print("Aggregation by Maximum")
print(max_df)
print("\n")

# remove levels
mean_df = mean_df.reset_index()
print("Removed levels")
print(mean_df)

  Team  salary
0   DS  100000
1   DT     100
2   DT   20000
3   DS       0
4   DS  100000


Aggregation by mean
            salary
Team              
DS    66666.666667
DT    10050.000000


Aggregation by mean
      salary
Team        
DS    200000
DT     20100


Aggregation by Median
      salary
Team        
DS    100000
DT     10050


Aggregation by Maximum
      salary
Team        
DS    100000
DT     20000


Removed levels
  Team        salary
0   DS  66666.666667
1   DT  10050.000000


Related - populating a column from a dictionary lookup:

In [9]:
# populating a dataframe column by dictionary lookup
mydict = {1: "Liping", 2: "Jordan", 3: "John", 4: "James"}
df = pd.DataFrame({"id": [3, 1, 2, 4, 10], "salary": [100000, 100, 20000, 0, 100000]})

df["name"]= pd.Series([mydict.get(id, "None") for id in df["id"]], index = df.index)
print(df)

   id  salary    name
0   3  100000    John
1   1     100  Liping
2   2   20000  Jordan
3   4       0   James
4  10  100000    None


_The code here uses a "series" to do a dictionary lookup. The command for the lookup is __dictionary__.get(). The brackets include the value we are looking up (in this case "id") and what to return if nothing is available (if the id is not in the dictionary). In this case, if the id is not in the dictionary the value will be listed as “None”. The last part just says this is a for loop for every “id” in the DataFrame's id column, and that we will use every row (index = df.index)_

## Working with DateTime Objects

In this last part we will use _pandas'_ date/time functionality for feature engineering:

In [10]:
df = pd.DataFrame({"id": [3, 1, 2, 4, 10], "salary": [100000, 100, 20000, 0, 100000], "dob": ["10/01/1950", "01/01/1990", "01/01/1980", "12/12/1981", "13/06/1981"]})
print(df)

# convert a column which has a date stored as string, into a column 
# stored as date. dayfirst=True means we use the UK style of date 
# (DDMMYYYY) rather than US (MMDDYYYY)
df['dob'] = pd.to_datetime(df['dob'], dayfirst=True)

# from a date object create cols for year, month, day, hour and minute
df['Year'] =  pd.DatetimeIndex(df['dob']).year
df['Month'] = pd.DatetimeIndex(df['dob']).month
df['Day'] = pd.DatetimeIndex(df['dob']).day
df['Hour'] = pd.DatetimeIndex(df['dob']).hour
df['Minute'] = pd.DatetimeIndex(df['dob']).minute

print("\n")
print(df)

# work out age by substracting the DOB from the date today
# note this is much harder than you may think it would be because years
# are different lengths and so ambiguous
df["Age"] = pd.to_datetime("today", dayfirst=True)-df['dob']
df["Age"] = round(df.Age.dt.days / 365, 1)

print("\n")
print(df)

   id  salary         dob
0   3  100000  10/01/1950
1   1     100  01/01/1990
2   2   20000  01/01/1980
3   4       0  12/12/1981
4  10  100000  13/06/1981


   id  salary        dob  Year  Month  Day  Hour  Minute
0   3  100000 1950-01-10  1950      1   10     0       0
1   1     100 1990-01-01  1990      1    1     0       0
2   2   20000 1980-01-01  1980      1    1     0       0
3   4       0 1981-12-12  1981     12   12     0       0
4  10  100000 1981-06-13  1981      6   13     0       0


   id  salary        dob  Year  Month  Day  Hour  Minute   Age
0   3  100000 1950-01-10  1950      1   10     0       0  72.1
1   1     100 1990-01-01  1990      1    1     0       0  32.1
2   2   20000 1980-01-01  1980      1    1     0       0  42.1
3   4       0 1981-12-12  1981     12   12     0       0  40.1
4  10  100000 1981-06-13  1981      6   13     0       0  40.6
